In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import string

In [3]:
filepath = "C:/Users/28chr/Downloads/dataset.xlsx"
data = pd.read_excel(filepath)
print(data)

                                                   text  label
0                                            oh my gosh    1.0
1     trouble sleeping, confused mind, restless hear...    1.0
2     All wrong, back off dear, forward doubt. Stay ...    1.0
3     I've shifted my focus to something else but I'...    1.0
4     I'm restless and restless, it's been a month n...    1.0
...                                                 ...    ...
6977                  I can't forget you #SpiritHadrian    0.0
6978  € ®šæœŸâ˜†ã€'..DJ DAIKI! DJ DAIKI! DJ DAIKI!.D...    0.0
6979                                          Dai5y! <3    0.0
6980  tired of clowns but still hopefully tonight if...    0.0
6981                                 MW SUBI WN LA VACA    0.0

[6982 rows x 2 columns]


In [8]:
print(data["text"])

0                                              oh my gosh
1       trouble sleeping, confused mind, restless hear...
2       All wrong, back off dear, forward doubt. Stay ...
3       I've shifted my focus to something else but I'...
4       I'm restless and restless, it's been a month n...
                              ...                        
6977                    I can't forget you #SpiritHadrian
6978    € ®šæœŸâ˜†ã€'..DJ DAIKI! DJ DAIKI! DJ DAIKI!.D...
6979                                            Dai5y! <3
6980    tired of clowns but still hopefully tonight if...
6981                                   MW SUBI WN LA VACA
Name: text, Length: 6982, dtype: object


In [41]:
num_anxious = 0
num_control = 0
num_chars_anxious = 0
num_chars_control = 0
num_punct_anxious = 0
num_punct_control = 0
num_cap_anxious = 0
num_cap_control = 0
training_upper = []
training_punct = []

In [42]:
word_list = {}
for i in range(len(data["text"])):
    # i % 5 == 4 is the test set
    if i % 5 != 4:
        post = str(data["text"][i])
        anxious = data["label"][i] == 1
        num_lower = 0
        num_upper = 0
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                num_lower += 1
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                num_upper += 1
                no_punct += (letter)
            else:
                no_punct += (" ")
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                if anxious:
                    word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                else:
                    word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                if anxious:
                    word_list[word] = (0, 1)
                else:
                    word_list[word] = (1, 0)
        num_punct = len(post) - num_upper - num_lower
        if anxious:
            num_anxious += 1
            num_chars_anxious += len(post)
            num_cap_anxious += num_upper
            num_punct_anxious += num_punct
        else:
            num_control += 1
            num_chars_control == len(post)
            num_cap_control += num_upper
            num_punct_control += num_punct
        training_upper.append(num_upper)
        training_punct.append(num_punct)

In [43]:
anxiety_scores = {}
for word in word_list:
    num_control = word_list[word][0]
    num_anxious = word_list[word][1]
    num_total = num_control + num_anxious
    # threshold to limit rarely used words; May vary but should be at least 1
    if num_total > 1:
        # consider squaring num_anxious so more frequently used words like restless have higher impact
        score = num_anxious / num_total
        anxiety_scores[word] = score

In [87]:
training_scores = []
training_labels = []
for i in range(len(data["text"])):
    if i % 5 != 4:
        caps = 0
        punct = 0
        one_offs = 0
        training_labels.append(data["label"][i])
        post = str(data["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs])

In [88]:
import math
for i in range(len(training_scores)):
    if math.isnan(training_scores[i][0]):
        training_scores[i][0] = 0
    if math.isnan(training_labels[i]):
        training_labels[i] = 0


In [89]:
training_scores = np.array(training_scores)
training_labels = np.array(training_labels)
reg = LinearRegression().fit(training_scores, training_labels)

In [94]:
res = []
for i in range(len(data["text"])):
    if i % 5 == 4:
        caps = 0
        punct = 0
        one_offs = 0
        label = data["label"][i]
        post = str(data["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs]]))
        res.append((out, label))

In [95]:
ind = 0
mini = 10000000000000000000 # arbitrarily big
for t in range (101):
    thres = t / 100
    mis = 0
    for tup in res:
        if tup[0][0] > thres and tup[1] == 0:
            mis += 1
        elif tup[0][0] < thres and tup[1] == 1:
            mis += 1
    if mis < mini:
        ind = t
        mini = mis
print(ind)
print(mini)

30
57


In [66]:
for t in range (21):
    thres = 0.2 + t / 100
    mis = 0
    n = 0
    for tup in res:
        if tup[0][0] > thres and tup[1] == 0:
            mis += 1
        elif tup[0][0] < thres and tup[1] == 1:
            mis += 1
        n += 1
    print(thres)
    print(mis)
    print(mis / n)
    print(n)
    print()

0.2
124
0.08882521489971347
1396

0.21000000000000002
113
0.08094555873925502
1396

0.22
105
0.07521489971346705
1396

0.23
99
0.07091690544412607
1396

0.24000000000000002
92
0.0659025787965616
1396

0.25
88
0.06303724928366762
1396

0.26
83
0.05945558739255014
1396

0.27
79
0.05659025787965616
1396

0.28
71
0.0508595988538682
1396

0.29000000000000004
67
0.04799426934097421
1396

0.30000000000000004
67
0.04799426934097421
1396

0.31
65
0.04656160458452722
1396

0.32
67
0.04799426934097421
1396

0.33
71
0.0508595988538682
1396

0.34
75
0.053724928366762174
1396

0.35
77
0.05515759312320917
1396

0.36
78
0.055873925501432664
1396

0.37
83
0.05945558739255014
1396

0.38
89
0.06375358166189111
1396

0.39
91
0.0651862464183381
1396

0.4
94
0.0673352435530086
1396



In [96]:
# misclassification rate using anxiety score, number of caps, number of punctuation symbols,
# length of post, and number of words that weren't used enough to have an anxiety score
57 / 1396 

0.04083094555873926

In [98]:
# Testing previous model using different parts of the dataset for testing
# Will help to determine optimal cutoff value based on first dataset
all_res = []
for j in range(5):
    num_anxious = 0
    num_control = 0
    word_list = {}
    for i in range(len(data["text"])):
        # i % 5 == j is the test set
        if i % 5 != j:
            post = str(data["text"][i])
            anxious = data["label"][i] == 1
            num_lower = 0
            num_upper = 0
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    num_lower += 1
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    num_upper += 1
                    no_punct += (letter)
                else:
                    no_punct += (" ")
            for word in no_punct.split(" "):
                word = word.lower()
                if word in word_list:
                    if anxious:
                        word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                    else:
                        word_list[word] = (word_list[word][0] + 1, word_list[word][1])
                else:
                    if anxious:
                        word_list[word] = (0, 1)
                    else:
                        word_list[word] = (1, 0)
            num_punct = len(post) - num_upper - num_lower
            if anxious:
                num_anxious += 1
            else:
                num_control += 1
    anxiety_scores = {}
    for word in word_list:
        num_control = word_list[word][0]
        num_anxious = word_list[word][1]
        num_total = num_control + num_anxious
        # threshold to limit rarely used words; May vary but should be at least 1
        if num_total > 1:
            # consider squaring num_anxious so more frequently used words like restless have higher impact
            score = num_anxious / num_total
            anxiety_scores[word] = score
    training_scores = []
    training_labels = []
    for i in range(len(data["text"])):
        if i % 5 != j:
            caps = 0
            punct = 0
            one_offs = 0
            training_labels.append(data["label"][i])
            post = str(data["text"][i])
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    no_punct += letter
                    caps += 1
                else:
                    no_punct += " "
                    punct += 1
            score = 0.0
            words = no_punct.split(" ")
            avg_divisor = len(words)
            for word in words:
                word = word.lower()
                if word in anxiety_scores:
                    score += anxiety_scores[word]
                else:
                    avg_divisor -= 1
                    one_offs += 1
            if avg_divisor == 0:
                score = 0
                avg_divisor = 1
            avg_score = score / avg_divisor
            training_scores.append([avg_score, caps, punct, len(post), one_offs])
    for i in range(len(training_scores)):
        if math.isnan(training_scores[i][0]):
            training_scores[i][0] = 0
        if math.isnan(training_labels[i]):
            training_labels[i] = 0
    training_scores = np.array(training_scores)
    training_labels = np.array(training_labels)
    reg = LinearRegression().fit(training_scores, training_labels)
    res = []
    for i in range(len(data["text"])):
        if i % 5 == j:
            caps = 0
            punct = 0
            one_offs = 0
            label = data["label"][i]
            post = str(data["text"][i])
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    no_punct += letter
                    caps += 1
                else:
                    no_punct += " "
                    punct += 1
            score = 0.0
            words = no_punct.split(" ")
            avg_divisor = len(words)
            for word in words:
                word = word.lower()
                if word in anxiety_scores:
                    score += anxiety_scores[word]
                else:
                    avg_divisor -= 1
                    one_offs += 1
            if avg_divisor == 0:
                score = 0
                avg_divisor = 1
            avg_score = score / avg_divisor
            out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs]]))
            res.append((out, label))
    all_res.append(res)


In [99]:
ind = 0
mini = 10000000000000000000 # arbitrarily big
for t in range (101):
    thres = t / 100
    mis = 0
    for res in all_res:
        for tup in res:
            if tup[0][0] > thres and tup[1] == 0:
                mis += 1
            elif tup[0][0] < thres and tup[1] == 1:
                mis += 1
    if mis < mini:
        ind = t
        mini = mis
print(ind)
print(mini)

30
331


In [100]:
# misclassification rate with one cutoff value based on all sample data
print(len(data["text"]))
print(331 / len(data["text"]))

6982
0.04740761959323976


In [204]:
# Combine all datasets
word_list = {}
for i in range(len(data["text"])):
    # i % 5 == 4 is the test set
    if i % 5 != 4:
        post = str(data["text"][i])
        anxious = data["label"][i] == 1
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += (letter)
            else:
                no_punct += (" ")
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                if anxious:
                    word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                else:
                    word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                if anxious:
                    word_list[word] = (0, 1)
                else:
                    word_list[word] = (1, 0)

In [185]:
word_list = {}

In [205]:
# control data from ds2
control_filepath = "C:/Users/28chr/Downloads/anon_control_tweets.tar.xz/anon_control_tweets/anon_control_tweets.csv"
data2 = pd.read_csv(control_filepath)

In [206]:
disorder_filepath = "C:/Users/28chr/Downloads/anon_disorder_tweets.tar.xz/anon_disorder_tweets/anon_disorder_tweets.csv"
data3 = pd.read_csv(disorder_filepath)

In [207]:
data2["text"] = data2["text"][:100000] 
data2["disorder"] = data2["disorder"][:100000]
data3["text"] = data3["text"][:100000]
data3["disorder"] = data3["disorder"][:100000]

In [208]:
# amount of data from data 2 and 3 limited to save time and space
for i in range(len(data2["text"])):
    if i % 5 != 4 and data2["disorder"][i] == "control": # dataset should be control only
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase or letter in string.ascii_uppercase:
                no_punct += letter
            else:
                no_punct += " "
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                word_list[word] = (1, 0)

In [209]:
for i in range(len(data3["text"])):
    if i % 5 != 4 and data3["disorder"][i] == "anxiety": # dataset should be anxiety only
        post = str(data3["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase or letter in string.ascii_uppercase:
                no_punct += letter
            else:
                no_punct += " "
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                word_list[word] = (word_list[word][0], word_list[word][1] + 1)
            else:
                word_list[word] = (0, 1)

In [210]:
anxiety_scores = {}
for word in word_list:
    num_control = word_list[word][0]
    num_anxious = word_list[word][1]
    num_total = num_control + num_anxious
    thresh = 1
    # threshold to limit rarely used words; May vary but should be at least 1
    if num_total > thresh:
        # consider squaring num_anxious so more frequently used words have higher impact
        score = num_anxious / num_total
        anxiety_scores[word] = score

In [211]:
neutral_words = []
scores_list = []
for word in anxiety_scores:
    scores_list.append(anxiety_scores[word])
avg = np.mean(scores_list)
std = np.std(scores_list)
lower = avg - std
upper = avg + std
for word in anxiety_scores:
    score = anxiety_scores[word]
    if score > lower and score < upper:
        neutral_words.append(word)
for word in neutral_words:
    anxiety_scores.pop(word)

In [192]:
training_scores = []
training_labels = []

In [217]:
training_scores = []
training_labels = []
for i in range(len(data["text"])):
    if i % 5 != 4:
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(data["label"][i])
        post = str(data["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])

In [218]:
for i in range(len(data2["text"])):
    if i % 5 != 4 and (data2["disorder"][i] == "control"):
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(0)
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])

In [219]:
for i in range(len(data3["text"])):
    if i % 5 != 4 and (data3["disorder"][i] == "anxiety"):
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(1)
        post = str(data3["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])

In [220]:
for i in range(len(training_scores)):
    if math.isnan(training_scores[i][0]):
        training_scores[i][0] = 0
    if math.isnan(training_labels[i]):
        training_labels[i] = 0

In [221]:
training_scores = np.array(training_scores)
training_labels = np.array(training_labels)
reg = LinearRegression().fit(training_scores, training_labels)

In [222]:
res = []
for i in range(len(data["text"])):
    if i % 5 == 4:
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        label = data["label"][i]
        post = str(data["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

In [223]:
for i in range(len(data2["text"])):
    if i % 5 == 4 and data2["disorder"][i] == "control":
        caps = 0
        punct = 0
        one_offs = 0
        label = 0
        neutral = 0
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

In [224]:
for i in range(len(data3["text"])):
    if i % 5 == 4 and data3["disorder"][i] == "anxiety":
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        label = 1
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

In [237]:
unc_divisor = 5
for u in range(21):
    uncertainty = u / 100
    ind = 0
    mini = 10000000000000000000 # arbitrarily big
    mini_mis = -1
    fp = 0
    fn = 0
    unc = 0
    for t in range (-100, 100):
        thres = t / 100
        mis = 0
        false_positive = 0
        false_negative = 0
        uncertain = 0
        for tup in res:
            if tup[0][0] > thres and tup[1] == 0:
                if tup[0][0] > thres + uncertainty:
                    mis += 1
                    false_positive += 1
                else:
                    uncertain += 1
            elif tup[0][0] < thres and tup[1] == 1:
                if tup[0][0] < thres - uncertainty:
                    mis += 1
                    false_negative += 1
                else:
                    uncertain += 1
        this_error = mis + (uncertain / unc_divisor)
        if this_error < mini:
            mini = this_error
            ind = t
            mini_mis = mis
            fn = false_negative
            fp = false_positive
            unc = uncertain
    print("Uncertainty:", uncertainty)
    print("Ideal threshold:", ind)
    print("Number misclassified:", mini_mis)
    print("Misclassification rate:", mini_mis / len(res))
    print("False positives:", fp)
    print("False negatives:", fn)
    print("Uncertain:", unc)
    print("Uncertainty rate:", unc / len(res))
    print("Error:", mini)
    print()

Uncertainty: 0.0
Ideal threshold: 30
Number misclassified: 14873
Misclassification rate: 0.3592859213450575
False positives: 5953
False negatives: 8920
Uncertain: 0
Uncertainty rate: 0.0
Error: 14873.0

Uncertainty: 0.01
Ideal threshold: 30
Number misclassified: 11332
Misclassification rate: 0.273746255676877
False positives: 3868
False negatives: 7464
Uncertain: 3541
Uncertainty rate: 0.0855396656681805
Error: 12040.2

Uncertainty: 0.02
Ideal threshold: 30
Number misclassified: 9437
Misclassification rate: 0.22796888588269398
False positives: 2852
False negatives: 6585
Uncertain: 5436
Uncertainty rate: 0.1313170354623635
Error: 10524.2

Uncertainty: 0.03
Ideal threshold: 29
Number misclassified: 7072
Misclassification rate: 0.1708377621026186
False positives: 2852
False negatives: 4220
Uncertain: 9682
Uncertainty rate: 0.23388733210938256
Error: 9008.4

Uncertainty: 0.04
Ideal threshold: 29
Number misclassified: 5508
Misclassification rate: 0.13305633394530872
False positives: 2355
Fa

In [166]:
res

[(array([0.77056579]), 1.0),
 (array([0.71967672]), 1.0),
 (array([0.71405665]), 1.0),
 (array([0.69855139]), 1.0),
 (array([0.72904995]), 1.0),
 (array([0.5339528]), 1.0),
 (array([0.72132411]), 1.0),
 (array([0.72493766]), 1.0),
 (array([0.79638272]), 1.0),
 (array([0.6872172]), 1.0),
 (array([0.75719561]), 1.0),
 (array([0.96415196]), 1.0),
 (array([0.65804156]), 1.0),
 (array([0.68723784]), 1.0),
 (array([0.72917988]), 1.0),
 (array([0.87392568]), 1.0),
 (array([0.96470977]), 1.0),
 (array([1.02240404]), 1.0),
 (array([0.56388042]), 1.0),
 (array([0.69258663]), 1.0),
 (array([0.82902798]), 1.0),
 (array([0.8633118]), 1.0),
 (array([0.70773412]), 1.0),
 (array([0.77182705]), 1.0),
 (array([0.58117992]), 1.0),
 (array([0.71676873]), 1.0),
 (array([0.789767]), 1.0),
 (array([0.67380284]), 1.0),
 (array([0.78342857]), 1.0),
 (array([0.71751422]), 1.0),
 (array([0.59536988]), 1.0),
 (array([0.61680735]), 1.0),
 (array([0.62872925]), 1.0),
 (array([0.80741192]), 1.0),
 (array([0.50304968